In [1]:
import os
import csv

samples = []
with open('./driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
samples[:5]

[['/Users/metrofun/Downloads/IMG/center_2019_04_08_23_47_53_973.jpg',
  ' /Users/metrofun/Downloads/IMG/left_2019_04_08_23_47_53_973.jpg',
  ' /Users/metrofun/Downloads/IMG/right_2019_04_08_23_47_53_973.jpg',
  '0.0',
  '0.0',
  '0',
  '5.7192'],
 ['/Users/metrofun/Downloads/IMG/center_2019_04_08_23_47_54_079.jpg',
  ' /Users/metrofun/Downloads/IMG/left_2019_04_08_23_47_54_079.jpg',
  ' /Users/metrofun/Downloads/IMG/right_2019_04_08_23_47_54_079.jpg',
  '0.0',
  '0.0',
  '0',
  '5.659069000000001'],
 ['/Users/metrofun/Downloads/IMG/center_2019_04_08_23_47_54_193.jpg',
  ' /Users/metrofun/Downloads/IMG/left_2019_04_08_23_47_54_193.jpg',
  ' /Users/metrofun/Downloads/IMG/right_2019_04_08_23_47_54_193.jpg',
  '0.0',
  '0.0',
  '0',
  '5.587722'],
 ['/Users/metrofun/Downloads/IMG/center_2019_04_08_23_47_54_304.jpg',
  ' /Users/metrofun/Downloads/IMG/left_2019_04_08_23_47_54_304.jpg',
  ' /Users/metrofun/Downloads/IMG/right_2019_04_08_23_47_54_304.jpg',
  '0.0',
  '0.0',
  '0',
  '5.528936'

In [2]:
from keras.preprocessing import image

def process_image(path):
    img = image.load_img(path.strip())
    return image.img_to_array(img)

Using TensorFlow backend.


In [3]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [4]:
import cv2
import numpy as np
import sklearn
from sklearn.utils import shuffle

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                try:
                    steering_center = float(batch_sample[3])
                except Exception as e:
                    print(steering_center)
                    print(e)
                    continue
                # create adjusted steering measurements for the side camera images
                correction = 0.3 # this is a parameter to tune
                steering_left = steering_center + correction
                steering_right = steering_center - correction

                # read in images from center, left and right cameras
                try:
                    img_center = process_image(batch_sample[0])
                    img_left = process_image(batch_sample[1])
                    img_right = process_image(batch_sample[2])
                except Exception as e:
                    print(e)
                    continue

                # add images and angles to data set
                images.extend((img_center, img_left, img_right))
                angles.extend((steering_center, steering_left, steering_right))
                
                images.extend((np.fliplr(img_center), np.fliplr(img_left), np.fliplr(img_right)))
                angles.extend((-steering_center, -steering_left, -steering_right))
                

            #TODO trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

In [5]:
from keras.layers import Input, Embedding, Lambda, Dense, BatchNormalization, Activation, Conv2D, Flatten, Cropping2D
from keras.models import Sequential

def build_model(nrows, ncols):
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(160, 320, 3)))
    model.add(Cropping2D(cropping=((60, 20), (0, 0)), data_format='channels_last'))
    model.add(Conv2D(24, (5,5), activation='relu', strides=(2,2)))
    model.add(Conv2D(36, (5,5), activation='relu', strides=(2,2)))
    model.add(Conv2D(48, (5,5), activation='relu', strides=(2,2)))
    model.add(Conv2D(64, (3,3), activation='relu', strides=(1,1)))
    model.add(Conv2D(64, (3,3), activation='relu', strides=(1,1)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='tanh'))
    return model

In [6]:
# Set our batch size
batch_size=32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

In [7]:
from math import ceil
from keras.callbacks import ModelCheckpoint

ch, row, col = 3, 80, 320  # Trimmed image format

model = build_model(row, col)

In [8]:
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [9]:
save_path = './weights-{epoch:03d}-val_loss-{val_loss:.5f}.hdf5'
checkpoint = ModelCheckpoint(save_path, monitor='val_loss', verbose=1, save_best_only=False, mode='min')
callbacks_list = [checkpoint]

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, \
            steps_per_epoch=ceil(len(train_samples)/batch_size), \
            validation_data=validation_generator, \
            validation_steps=ceil(len(validation_samples)/batch_size), \
            callbacks=callbacks_list, \
            epochs=10, verbose=1)

Epoch 1/10
14/14 [==============================] - 32s 2s/step - loss: 0.0944 - val_loss: 0.0756

Epoch 00001: saving model to ./weights-001-val_loss-0.07557.hdf5
Epoch 2/10
14/14 [==============================] - 31s 2s/step - loss: 0.0743 - val_loss: 0.0662

Epoch 00002: saving model to ./weights-002-val_loss-0.06618.hdf5
Epoch 3/10
14/14 [==============================] - 32s 2s/step - loss: 0.0677 - val_loss: 0.0665

Epoch 00003: saving model to ./weights-003-val_loss-0.06648.hdf5
Epoch 4/10
14/14 [==============================] - 33s 2s/step - loss: 0.0615 - val_loss: 0.0598

Epoch 00004: saving model to ./weights-004-val_loss-0.05985.hdf5
Epoch 5/10
14/14 [==============================] - 32s 2s/step - loss: 0.0544 - val_loss: 0.0604

Epoch 00005: saving model to ./weights-005-val_loss-0.06038.hdf5
Epoch 6/10
14/14 [==============================] - 32s 2s/step - loss: 0.0499 - val_loss: 0.0556

Epoch 00006: saving model to ./weights-006-val_loss-0.05563.hdf5
Epoch 7/10
14/14